In [6]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [7]:
!ls -lh

total 24G
-rw-rw-r-- 1 core core  3.2G Aug 13 11:40 all_models_team_submission_v1_large_20190813.csv
-rw-rw-r-- 1 core core  850M Aug 22 14:21 all_models_team_submission_v1_large_20190813.csv.zip
drwxrwxr-x 4 core core  4.0K Aug 10 15:30 catboost_info
-rw-rw-r-- 1 core core   852 Aug  4 15:31 cat_feature_index.c1a4dabd-715a703f-75517141-d81266d1.tmp
-rw-rw-r-- 1 core core   852 Aug  4 15:04 cat_feature_index.e36fdba6-6cd399f6-229ea45f-ea07b906.tmp
-rw-rw-r-- 1 core core  2.2M Aug 20 15:31 detect-57-ensemble.ipynb
-rw-rw-r-- 1 core core  2.2M Aug 22 16:05 detect-57-ensemble-level2.ipynb
-rw-rw-r-- 1 core core  1.3M Aug 21 07:32 eda.ipynb
-rw-rw-r-- 1 core core  421M Aug  8 09:52 ensemble_detect_57_0808_1.csv
-rw-rw-r-- 1 core core  268M Aug  8 09:56 ensemble_detect_57_0808_1_top50.csv
-rw-rw-r-- 1 core core  421M Aug  8 15:04 ensemble_detect_57_0808_2_with_weights.csv
-rw-rw-r-- 1 core core  399M Aug 19 17:09 ensemble_detect_57_0819_1.csv
-rw-rw-r-- 1 core core  399M Aug 19 17:10 ensemb

In [36]:
csv_files = [
            #'sub_sort_200_0820_2_fast_ensemble.csv',
            #'../vrd/lb23578/sub_tuple_0821_4.csv',
            #'../vrd/lb25388/sub_tuple_0825_cdiff.csv',
            '../vrd/lb25388/sub_tuple_0825_cdiff_exclude_inside_of.csv',
            '../vrd/insideof/sub_insideof.csv',
            'ensemble_is_42_0808_1_3_models.csv'
            #'sub_is_42_0726_2.csv',
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [37]:
for df in dfs:
    df.PredictionString = df.PredictionString.fillna('')

In [38]:
len(dfs[1].iloc[0].PredictionString)

0

In [39]:
# lower conf for is model, not working
# dfs[1].PredictionString = dfs[1].PredictionString.map(lambda x: get_low_conf_pred_str(x))

In [40]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500165 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,0.114314 /m/04yx4 0.9498244 0.3313462 0.999023...
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,0.178867 /m/01_5g 0.7968998 0.0202855 0.997210...


In [89]:
#keep top 100
#dfs[0].PredictionString = dfs[0].PredictionString.map(lambda x: ' '.join(x.split(' ')[:1200]))

In [42]:
dfs[1].head(10)

,ImageId,PredictionString
0,21135d5ed0e7a0b7,
1,763c72626bf24534,
2,7540c162180973b1,
3,c4e522dce52cb02f,
4,cafcd9496c808085,
5,53a6a4e7561a8a51,0.033577 /m/04yx4 0.335658 0.3165081 0.6516815...
6,52d6a184cddab056,
7,4ddce90e2f1625eb,
8,76a33fc3a2b40714,0.044692 /m/04yx4 0.3503682 0.4429366 0.393902...
9,fc98e22362caea6d,


In [43]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString
    else:
        x = x.str.cat(d.PredictionString, sep=' ')

In [44]:
dfs[0].PredictionString = x
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

32572 0


In [45]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.500165 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,0.114314 /m/04yx4 0.9498244 0.3313462 0.999023...
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,0.178867 /m/01_5g 0.7968998 0.0202855 0.997210...


In [46]:
dfs[0].to_csv('merge_0828_1_insideof.csv.zip', index=False, compression='zip')

In [47]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f merge_0828_1_insideof.csv.zip -m "merge insideof model"

100%|████████████████████████████████████████| 484M/484M [00:23<00:00, 21.3MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [ ]:
!ls

In [11]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

10336

In [56]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

0

In [12]:
len(dfs[1].loc[dfs[1].PredictionString.isna()])

41

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [4]:
df_is = pd.read_csv('ensemble_is_42_0808_1_3_models.csv')
df_is.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.1689330 /m/04bcr3 0.3664692 0.7250217 0.4814...
1,763c72626bf24534,0.7671249 /m/04bcr3 0.6873273 0.8193369 0.9977...
2,7540c162180973b1,0.0219995 /m/04dr76w 0.0762036 0.3234025 0.120...
3,c4e522dce52cb02f,0.4940703 /m/04bcr3 0.8307936 0.6809913 0.9920...
4,cafcd9496c808085,0.0129044 /m/04dr76w 0.0000000 0.0600080 0.031...


In [5]:
def get_low_conf_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return pred_str
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    
    res = []
    for i, c in enumerate(det_items):
        if i % 12 == 0:
            res.append(str(float(c)/4))
        else:
            res.append(c)
    return ' '.join(res)